In [1]:
from core import *
from core_mps import *
from quantum_plots import *
from mps.mpo import MPO, MPOList

Global props:
fontsizes :
     tiny :  6
     scriptsize :  8
     footnotesize :  9
     small :  10
     normalsize :  11
     large :  12
     Large :  14
     LARGE :  17
     huge :  20
     Huge :  25
colors :
     quantumviolet :  #53257F
     quantumgray :  #555555


## Time evolution

### a) Ladder operators

We want to construct an operator that maps a binary number $s=s_1s_2\ldots s_m$ to $s+1$ or $s-1$. Let us begin with the operator $S^+$ which will increase the value of the register by one. The table of additions is

| a | b | a+b | c |
|---|---|-----|---|
| 0 | 0 |  0  | 0 |
| 0 | 1 |  1  | 0 |
| 1 | 0 |  1  | 0 |
| 1 | 1 |  0  | 1 |

We can implement this with a tensor $A_{cb}^{a',a}$ that is 1 only on the values of that table.

In [2]:
def mpoSup(n, **kwdargs):
    
    A = np.zeros((2,2,2,2))
    A[0,0,0,0] = 1.
    A[0,1,1,0] = 1.
    A[1,0,1,1] = 1.
    A[0,1,0,1] = 1.
    
    R = A[:,:,:,[1]]
    L = A[[0],:,:,:] # + A[[1],:,:,:]
    return MPO([L] + [A] * (n-2) + [R], **kwdargs)

Similarly, we would have another tensor for the decrease

| a | b | a-b | c |
|---|---|-----|---|
| 0 | 0 |  0  | 0 |
| 0 | 1 |  1  | 1 |
| 1 | 0 |  1  | 0 |
| 1 | 1 |  0  | 0 |

In [3]:
def mpoSdown(n, **kwdargs):
    
    A = np.zeros((2,2,2,2))
    A[0,0,0,0] = 1.
    A[0,1,1,0] = 1.
    A[0,0,1,1] = 1.
    A[1,1,0,1] = 1.
    
    R = A[:,:,:,[1]]
    L = A[[0],:,:,:] # + A[[1],:,:,:]
    return MPO([L] + [A] * (n-2) + [R], **kwdargs)

And finally, if we want to make a superposition of both changes
$$O = \epsilon_0 + \epsilon_1 S^+ + \epsilon_2 S^-,$$
we can do it easily with bond dimension 3.

In [4]:
def mpo_combined(n,a,b,c, **kwdargs):
    
    A = np.zeros((3,2,2,3))
    # Internal bond dimension 0 is nothing, 1 is add 1, 2 is subtract 1
    
    A[0,0,0,0] = 1.
    A[0,1,1,0] = 1.
    # Increase
    A[0,1,0,1] = 1.
    A[1,0,1,1] = 1.
    # Decrease
    A[2,1,0,2] = 1.
    A[0,0,1,2] = 1.
    
    R = a*A[:,:,:,[0]] + b*A[:,:,:,[1]] + c*A[:,:,:,[2]]
    L = A[[0],:,:,:] # + A[[1],:,:,:] + A[[2],:,:,:]
    return MPO([L] + [A] * (n-2) + [R], **kwdargs)

We can see that the operators are the tridiagonal matrices we expect

In [5]:
mpoSup(3).tomatrix()

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.]])

In [6]:
mpoSdown(3).tomatrix()

array([[0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [7]:
mpo_combined(3, 1, 2, 3).tomatrix()

array([[1., 3., 0., 0., 0., 0., 0., 0.],
       [2., 1., 3., 0., 0., 0., 0., 0.],
       [0., 2., 1., 3., 0., 0., 0., 0.],
       [0., 0., 2., 1., 3., 0., 0., 0.],
       [0., 0., 0., 2., 1., 3., 0., 0.],
       [0., 0., 0., 0., 2., 1., 3., 0.],
       [0., 0., 0., 0., 0., 2., 1., 3.],
       [0., 0., 0., 0., 0., 0., 2., 1.]])

In [8]:
from mps.truncate import simplify

def apply_all(mpos, ψmps, canonical=True, tolerance=DEFAULT_TOLERANCE, normalize=True, debug=[]):
    def multiply(A, B):
        C = np.einsum('aijb,cjd->acibd',A,B)
        s = C.shape
        return C.reshape(s[0]*s[1],s[2],s[3]*s[4])
    err = 0.
    for (i,mpo) in enumerate(reversed(mpos)):
        ψmps = MPS([multiply(A,B) for A,B in zip(mpo,ψmps)])
        if canonical:
            newψmps, theerr, _ = simplify(ψmps, tolerance=tolerance, normalize=normalize)
            theerr = np.sqrt(theerr)
            if 'norm' in debug:
                print(f'Initial state norm {ψmps.norm2()}, final {newψmps.norm2()}')  
            elif 'trunc' in debug:
                n1 = ψmps.norm2()
                sc = abs(mps.expectation.scprod(ψmps, newψmps))
                n2 = newψmps.norm2()
                real_err = np.sqrt(2*np.abs(1.0 - sc/np.sqrt(n1*n2)))
                D = max(A.shape[-1] for A in ψmps)
                print(f'error={theerr:5g}, estimate={np.sqrt(real_err):5g}, norm={n1:5f}, after={n2:3f}, D={D}')
            err += theerr
            ψmps = newψmps
            newψmps = None
    return ψmps, err

### b) Fokker-Planck equation

Let us assume a variable that follows a Wiener process $W$ in the Ito representation
$$dX = \mu(X,t)dt + \sigma(X,t) dW.$$

The probability distribution for the random variable $X$ evolves as
$$\frac{\partial}{\partial t}p(x,t) = -\frac{\partial}{\partial x} \left[\mu(x,t)p(x,t)\right] + \frac{\partial^2}{\partial x^2}[D(x,t)p(x,t)].$$

We are going to use a finite-difference solver for this equation, with the following approximations
$$\frac{\partial}{\partial x}f(x) = \frac{f(x+\delta)-f(x-\delta)}{2\delta} + \mathcal{O}(\delta^2),$$
$$\frac{\partial^2}{\partial x^2}f(x) = \frac{f(x+\delta)+f(x-\delta)-2f(x)}{\delta^2} + \mathcal{O}(\delta).$$

Assuming constant drift and diffusion and labelling $p(x_s,t) = p_s(t),$ we have
$$p_s(t+\delta t) = p_s(t) + \delta t \left[\mu\frac{p_{s-1}(t)-p_{s+1}(t)}{2\delta{x}}
+ D \frac{p_{s+1}(t)+p_{s-1}(t)-2p_s(t)}{\delta{x}^2}\right].$$

In terms of our ladder operators,
$$\vec{p}(t+\delta t) = \left(1-2\delta{t}\frac{D}{\delta{x}^2}\right)\vec{p}
+\delta t\left(-\frac{\mu}{2\delta{x}}+\frac{D}{\delta{x}^2}\right)S^+\vec{p}
+\delta t\left(+\frac{\mu}{2\delta{x}}+\frac{D}{\delta{x}^2}\right)S^-\vec{p}.$$

But this equation blows exponentially unless $\delta tD/\delta x^2 \ll 1.$

An alternative is to write
$$\frac{p(t+\delta t)-p(t)}{\delta t} = \frac{1}{2}\hat{G}\left[p(t+\delta t)+p(t)\right],$$
leading to
$$\left(1-\frac{\delta t}{2}\hat{G}\right) p(t+\delta t) = \left(1 + \frac{\delta t}{2}\hat{G}\right)p(t),$$
and the numerically stable solution
$$ p(t+\delta t) = \left(1-\frac{\delta t}{2}\hat{G}\right)^{-1}\left(1 + \frac{\delta t}{2}\hat{G}\right)p(t).$$

The following operator implements the MPO $(1+\delta{t}\hat{G}).$ The sign and factors $\tfrac{1}{2}$ can be changed by simply changing $\delta{t}.$

In [9]:
def mpo_drift(n, δt, δx, μ, D, **kwdargs):
    Dx = D/δx**2
    μx = μ/(2*δx)
    a = 1 - 2*δt*Dx
    b = δt*(Dx-μx)
    c = δt*(Dx+μx)
    print(f'δx={δx}, δt={δt}, D={D}, μ={μ}')
    print(f'Coefficients: a={a}, b={b}, c={c}')
    return mpo_combined(n, a, b, c, **kwdargs)

We test these methods first with wavefunctions

In [10]:
import scipy.sparse as sp
import os.path
from mps.truncate import cgs

def FokkerPlanck(N, σ, T, steps, b=None, a=None, μ=0.0, D=1.0, filename=None):

    if b is None:
        b = 7*σ
    if a is None:
        a = -b
    δx = (b-a)/2**N
    times = np.linspace(0, T, steps)
    δt = times[1]
    ψmps0 = GaussianMPS(N, σ, a=a, b=b, GR=False, canonical=True, normalize=True)
    ψ0 = ψmps0.tovector()
    x = np.linspace(a, b, 2**N)

    mpo1 = mpo_drift(N, 0.5*δt, δx, μ, D, simplify=False)
    mpo2 = mpo_drift(N, -0.5*δt, δx, μ, D, simplify=False)
    op1 = sp.csr_matrix(mpo1.tomatrix())
    op2 = sp.csr_matrix(mpo2.tomatrix())
    ψ = [ψ0]
    print(f'int(ψ)={np.sum(ψ0)}, |ψ|={np.linalg.norm(ψ0)}')
    for t in times[1:]:
        ψ0 = sp.linalg.spsolve(op2, op1 @ ψ0)
        n0 = np.linalg.norm(ψ0)
        ψmps0, err = mps.truncate.cgs(mpo2, mpo1.apply(ψmps0))
        ψ1 = ψmps0.tovector()
        n1 = np.linalg.norm(ψ1)
        sc = 1 - np.vdot(ψ1, ψ0)/(n1*n0)
        print(f'int(ψ)={np.sum(ψ0):5f}, |ψ|={n0:5f}, |ψmps|={n1:5f}, sc={sc:5g}, err={err:5g}')
        ψ.append(ψ1)

    if filename is not None:
        with open(filename,'wb') as f:
            pickle.dump((ψ, x, times, D, μ, b), f)
    return ψ, x, times

if not os.path.exists('data/fokker-planck-2d-a.pkl'):
    FokkerPlanck(10, 1.0, 10, 100, μ=-0.2, D=0.1, b=10, filename='data/fokker-planck-2d-a.pkl');